In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
# timm 包 PyTorchImageModels，包含了大量的预训练模型和工具集
from timm import create_model

In [ ]:
# Fastai 库，Pytorch 的上层封装，可以简化训练模型所需要的代码
from fastai.vision.all import *

In [ ]:
# 创建随机种子
set_seed(999, reproducible=True)
# 每批次数据量
BATCH_SIZE = 8                       

In [ ]:
# 数据集位置
dataset_path = Path('../input/petfinder-pawpularity-score/')
# 显示有哪些数据集
dataset_path.ls()

In [ ]:
# 读取训练集 CSV
train_df = pd.read_csv(dataset_path/'train.csv')
# 展示一部分
train_df.head()

In [ ]:
# 将相对地址写入到 dataframe 中
train_df['path'] = train_df['Id'].map(lambda x:str(dataset_path/'train'/x)+'.jpg')
# ID 列已不需要，删除
train_df = train_df.drop(columns=['Id'])
# 清洗打乱数据
# sample(按比例随机 frac 返回一组打乱的数据，并重建索引)
train_df = train_df.sample(frac=1).reset_index(drop=True)
# 展示部分打乱的数据
train_df.head()

In [ ]:
# 展示训练集的数据量
len_df = len(train_df)
print(f"There are {len_df} images")

In [ ]:
# 展示分数的平均、中位数、标准偏差
train_df['Pawpularity'].hist(figsize = (10, 5))
print(f"The mean Pawpularity score is {train_df['Pawpularity'].mean()}")
print(f"The median Pawpularity score is {train_df['Pawpularity'].median()}")
print(f"The standard deviation of the Pawpularity score is {train_df['Pawpularity'].std()}")

In [ ]:
# 统计有多少不同的分数
print(f"There are {len(train_df['Pawpularity'].unique())} unique values of Pawpularity score")

In [ ]:
# 归一化分数
train_df['norm_score'] = train_df['Pawpularity']/100
# 显示归一化之后的结果
train_df['norm_score']

In [ ]:
# 打开图片并查看
im = Image.open(train_df['path'][1])
# 获取宽度和高度
width, height = im.size
print(width,height)

In [ ]:
im

In [ ]:
# 将预训练模型保存到此文件夹后面进行加载
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
    os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'


In [ ]:
# 设定随机数种子
seed=999
set_seed(seed, reproducible=True)
# 固定 cuda、torch 中的随机数
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
# 固定卷积算法（使用确定性算法，保证无论重复多少次网络输入和输出都一样）
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

In [ ]:
# 斯透奇斯规则
# 经验公式（这个是试出来的，没有什么理论），用于计算直方图的分组数量
num_bins = int(np.floor(1+(3.3)*(np.log2(len(train_df)))))
# num_bins

In [ ]:
# 根据规则绘制直方图
train_df['bins'] = pd.cut(train_df['norm_score'], bins=num_bins, labels=False)
train_df['bins'].hist()

In [ ]:
# 这个没用到
from sklearn.model_selection import KFold
# 分层采样交叉切分训练集和数据集，确保训练集，测试集中各类别样本的比例与原始数据集中相同。
from sklearn.model_selection import StratifiedKFold

# 初始化赋值，每一个单元格默认 -1
train_df['fold'] = -1

# 折数，分为 10 份数据
N_FOLDS = 10
# 获取分好后的数据（n_splits 是拆分的份数，random_state 是随机数种子，shuffle 是是否需要打乱数据
strat_kfold = StratifiedKFold(n_splits=N_FOLDS, random_state=seed, shuffle=True)
# 从分好的数据中进行抽样分组
for i, (_, train_index) in enumerate(strat_kfold.split(train_df.index, train_df['bins'])):
    # 设定每个数据的 fold 组
    train_df.iloc[train_index, -1] = i
    
# 重设列元素数据类型为 int
train_df['fold'] = train_df['fold'].astype('int')
# 统计每个采样组的数据量并画图
train_df.fold.value_counts().plot.bar()

In [ ]:
# 训练集的第 0 组数据（分了 10 组）查看一部分
train_df[train_df['fold']==0].head()

In [ ]:
# 查看训练集第 0 组数据的统计数量
train_df[train_df['fold']==0]['bins'].value_counts()

In [ ]:
# 查看训练集第 1 组数据的统计数量
train_df[train_df['fold']==1]['bins'].value_counts()

In [ ]:
# 按照题目给出的计算公式编写误差函数
def petfinder_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

In [ ]:
# 用于返回数据的函数，传入值为此时是第几组作为验证集（组：fold）
def get_data(fold):
    # 拷贝训练集用来做操作，不改变原始数据（因为要多次调用训练集，每次用于测验的数据组都不同）
    train_df_f = train_df.copy()
    # 标记训练集中多少用于做测试
    train_df_f['is_valid'] = (train_df_f['fold'] == fold)
    # 创建 dataloader（from_df 表示从 dataframe 中加载数据）
    dls = ImageDataLoaders.from_df(train_df_f, # 训练集
                               valid_col='is_valid', # 分配判断验证数据的列
                               seed=999, # 随机数
                               fn_col='path', # 文件名列
                               label_col='norm_score', # 标签所在的列
                               y_block=RegressionBlock, # 标签的类型（默认有单标签分类，多标签分类，回归）
                               bs=BATCH_SIZE, # 批次大小
                               num_workers=8, # 载入线程
                               item_tfms=Resize(224), # transforms 配置（在批次转换之前进行处理，调整大小为 224 * 224 的图片，对应模型输入）
                               # 批次中 transforms 配置（因为此处是检测受欢迎程度，因此图像不使用剪裁旋转等方式增强处理）
                               batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()])) 
    
    return dls


In [ ]:
# 验证数据集的数量是否正确
the_data = get_data(0)
# 断 言(异常处理)
assert (len(the_data.train) + len(the_data.valid)) == (len(train_df)//BATCH_SIZE)

In [ ]:
# 创建训练过程
def get_learner(fold_num):
    # 获取 input 数据
    data = get_data(fold_num)
    # 创建模型（使用预训练的模型和权重）
    model = create_model('swin_large_patch4_window7_224', pretrained=True, num_classes=data.c)
    # 创建学习过程，使用 fastai 自带的 BCEWithLogitsLossFlat 做损失函数
    learn = Learner(data, model, loss_func=BCEWithLogitsLossFlat(), metrics=petfinder_rmse).to_fp16()
    # 返回训练器
    return learn

In [ ]:
# 读取测试集的数据并显示部分
test_df = pd.read_csv(dataset_path/'test.csv')
test_df.head()

In [ ]:
# 测试集
# [1]*len(test_df) 和直接赋值 1 是一样的
test_df['Pawpularity'] = [1]*len(test_df)
# 新建地址列表
test_df['path'] = test_df['Id'].map(lambda x:str(dataset_path/'test'/x)+'.jpg')
# 删除 ID
test_df = test_df.drop(columns=['Id'])
# 归一化分数
train_df['norm_score'] = train_df['Pawpularity']/100

In [ ]:
# 搜索学习率（返回一个不同学习率对于 loss 的变化曲线，寻找目标是变化最大的一个点）
get_learner(fold_num=0).lr_find(end_lr=3e-2)

In [ ]:
# 垃圾回收
import gc

In [ ]:
# 用于保存预测结果
all_preds = []
# 利用 K 折分数据的数量，设定训练的总次数
for i in range(N_FOLDS):
    # 输出现在的循环序号
    print(f'Fold {i} results')
    # 初始化训练和测试数据集
    learn = get_learner(fold_num=i)
    # 开始训练（5 个 epoch，2e-5 作为学习率
    # cbs 中是关于训练时配置的参数
    learn.fit_one_cycle(5, 2e-5, cbs=[SaveModelCallback(), EarlyStoppingCallback(monitor='petfinder_rmse', comp=np.less, patience=2)]) 
    # 画出训练时记录的 loss 变化曲线
    learn.recorder.plot_loss()
    # 重新加载 dataloader
    dls = ImageDataLoaders.from_df(train_df, # 重新传入训练集
                               valid_pct=0.2, # 按 20% 的比例分离训练集和验证集
                               seed=999,  # 随机数种子
                               fn_col='path', # 从 path 列中读取数据
                               label_col='norm_score', # 从归一化的分数中读取数据作为标签
                               y_block=RegressionBlock, # 设定标签为回归
                               bs=BATCH_SIZE, # 设定批次
                               num_workers=8, # 多线程加载
                               item_tfms=Resize(224), # transforms 配置（在批次转换之前进行处理，调整大小为 224 * 224 的图片，对应模型输入）
                               # 批次中 transforms 配置（因为此处是检测受欢迎程度，因此图像不使用剪裁旋转等方式增强处理）
                               batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()])) 
    # 测试集 dataloader
    test_dl = dls.test_dl(test_df)
    # 测试集增强（n 为测试次数，多次测试取平均值）
    preds, _ = learn.tta(dl=test_dl, n=5, beta=0)
    # 把测试结果加入数组
    all_preds.append(preds)
    # 删除当前批的训练器
    del learn
    # 清除 cuda 缓存
    torch.cuda.empty_cache()
    # 垃圾回收
    gc.collect()

In [ ]:
# 显示测试结果
all_preds

In [ ]:
# 拿到提交所需要的 csv 文件
sample_df = pd.read_csv(dataset_path/'sample_submission.csv')
# 取预测的平均值（一共 10 组，取平均）
preds = np.mean(np.stack(all_preds), axis=0)
# 生成结果列
sample_df['Pawpularity'] = preds*100
# 生成结果 csv 文件
sample_df.to_csv('submission.csv',index=False)

In [ ]:
# 读取结果文件查看效果
pd.read_csv('submission.csv').head()